<a href="https://colab.research.google.com/github/arunm917/Climate-Action-Tool/blob/main/solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading packages and files

In [ ]:
import numpy as np
import pandas as pd
import gdown
import regex as re
from scipy.optimize import minimize

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# # downloading file from gdrive
# output = 'equations'
# file_id = '1q9cRaPAXvg2KSuc9f8WDjnfLQDFdmtew' # Google drive ID
# #Download the file
# gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
# print('\nDONE.')

In [ ]:
# downloading file from gdrive
output = 'equations'
file_id = '1raendJQeKuzTduzPYIgcIXZYlRA7esAU' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1raendJQeKuzTduzPYIgcIXZYlRA7esAU
To: /content/equations
100%|██████████| 797/797 [00:00<00:00, 2.53MB/s]


DONE.


In [ ]:
# downloading file from gdrive
output = 'parameters'
file_id = '1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K
To: /content/parameters
100%|██████████| 433/433 [00:00<00:00, 1.46MB/s]


DONE.


In [ ]:
# downloading file from gdrive
output = 'variables'
file_id = '1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ
To: /content/variables
100%|██████████| 236/236 [00:00<00:00, 694kB/s]


DONE.


# Processing files

In [ ]:
parameters_df = pd.read_csv('parameters')

In [ ]:
parameters_df

,parameters,values,type
0,phi_crusher,3.50,fixed
1,phi_rm,26.00,fixed
2,phi_cm,76.30,fixed
3,phi_phk,37.70,fixed
4,phi_cooler,37.70,fixed
5,phi_cementmill,30.00,fixed
6,w_add_ls,0.11,fixed
7,w_c_ls,0.06,fixed
8,w_c_coal,0.76,fixed
9,w_cl_rm,0.64,fixed


In [ ]:
correct_vars_df = parameters_df.loc[parameters_df['type'] == 'correct', ['parameters','values']]

In [ ]:
correct_vars_df

,parameters,values
15,eta_plant,0.6


In [ ]:
variables_df = pd.read_csv('variables')
variables_df

,variables,values
0,m_ls,100
1,m_rm,0
2,m_rawmeal,0
3,m_pcoal,0
4,m_phk_air,0
5,m_hotclinker,0
6,m_stackgas,0
7,m_clinker,0
8,m_cement,0
9,m_cooler_air,0


In [ ]:
'''The variables are split into decision variables and static variables. Variables whose values are specified
   by the user are static variables and the ones whose values are set to 0 are set as decision variables. Only 
   the decision variables are used by the optimization function. Static variables get substituted by their values.'''

static_var_df = variables_df[variables_df['values'] != 0]
decision_var_df = variables_df[variables_df['values'] == 0]

In [ ]:
static_var_list = list(static_var_df['variables'])
decision_var_list = list(decision_var_df['variables'])

In [ ]:
''' This line of code checks if the parameter is set to fixed, float or correctable.
    fixed = parameter value is fixed
    float = parameter is part of the decision variable. Value of parameter is discarded
    correct = parameter is changed by a small margin by making it a decision variable and applying a tolerance (using bounds).
    Parameter value is not discarded.'''
    
float_vars = parameters_df.loc[parameters_df['type'].isin(['float']), 'parameters'].tolist()
decision_var_list.extend(float_vars)
correct_vars = parameters_df.loc[parameters_df['type'].isin(['correct']), 'parameters'].tolist()
decision_var_list.extend(correct_vars)
parameters_df = parameters_df[~parameters_df['parameters'].isin(float_vars)]

In [ ]:
correct_vars

['eta_plant']

In [ ]:
parameters_df

,parameters,values,type
0,phi_crusher,3.50,fixed
1,phi_rm,26.00,fixed
2,phi_cm,76.30,fixed
3,phi_phk,37.70,fixed
4,phi_cooler,37.70,fixed
5,phi_cementmill,30.00,fixed
6,w_add_ls,0.11,fixed
7,w_c_ls,0.06,fixed
8,w_c_coal,0.76,fixed
9,w_cl_rm,0.64,fixed


In [ ]:
print(decision_var_list)

['m_rm', 'm_rawmeal', 'm_pcoal', 'm_phk_air', 'm_hotclinker', 'm_stackgas', 'm_clinker', 'm_cement', 'm_cooler_air', 'e_crusher', 'e_rawmill', 'e_coalmill', 'e_phk_total', 'e_cooler', 'e_cementmill', 'm_co2', 'eta_kiln', 'eta_plant']


In [ ]:
with open('equations', 'r') as f:
    # Read the equations line by line
    eq_lines = f.readlines()

In [ ]:
print(eq_lines)

['eq1: (1 + w_add_ls) * m_ls - m_rm\n', 'eq2: m_rm - m_rawmeal\n', 'eq3: (beta_phk * w_cl_ls / gcv) * (m_ls / eta_plant) - m_pcoal  \n', 'eq4: m_cooler_air - w_c_coal * 23.0158 * m_pcoal \n', 'eq5: m_cooler_air - m_phk_air\n', 'eq6: (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas\n', 'eq7: w_cl_rm * m_rawmeal + (1 - w_c_coal) * m_pcoal - m_clinker\n', 'eq8: m_hotclinker - m_clinker\n', 'eq9: m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement\n', 'eq10: (m_ls * mol_wt_co2 / 100) + (w_c_coal * eta_kiln * mol_wt_co2 / mol_wt_c) * m_pcoal - m_co2 \n', 'eq11: e_crusher - phi_crusher * m_rm\n', 'eq12: e_rawmill - phi_rm * m_rawmeal\n', 'eq13: e_coalmill - phi_cm * m_pcoal\n', 'eq14: e_phk_total - beta_phk * 1000 * m_clinker \n', 'eq15: e_cooler - phi_cooler * m_hotclinker\n', 'eq16: e_cementmill - phi_cementmill * m_cement\n']


In [ ]:
# Create a list to store the equations
eq_list = []

# Loop through the equation lines
for eq_line in eq_lines:
    # Split the line into the equation name and the equation expression
    eq_name, eq_expr = eq_line.strip().split(':')
    # Convert the tuple of symbols to a single expression
    eq_list.append(eq_expr) 
    # Add the equation to the dictionary
print(eq_list)

[' (1 + w_add_ls) * m_ls - m_rm', ' m_rm - m_rawmeal', ' (beta_phk * w_cl_ls / gcv) * (m_ls / eta_plant) - m_pcoal', ' m_cooler_air - w_c_coal * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas', ' w_cl_rm * m_rawmeal + (1 - w_c_coal) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement', ' (m_ls * mol_wt_co2 / 100) + (w_c_coal * eta_kiln * mol_wt_co2 / mol_wt_c) * m_pcoal - m_co2', ' e_crusher - phi_crusher * m_rm', ' e_rawmill - phi_rm * m_rawmeal', ' e_coalmill - phi_cm * m_pcoal', ' e_phk_total - beta_phk * 1000 * m_clinker', ' e_cooler - phi_cooler * m_hotclinker', ' e_cementmill - phi_cementmill * m_cement']


In [ ]:
# Creating dictionary for parameters and their values
param_dict = dict(zip(parameters_df['parameters'],parameters_df['values']))
print(param_dict)

{'phi_crusher': 3.5, 'phi_rm': 26.0, 'phi_cm': 76.3, 'phi_phk': 37.7, 'phi_cooler': 37.7, 'phi_cementmill': 30.0, 'w_add_ls': 0.11, 'w_c_ls': 0.06, 'w_c_coal': 0.76, 'w_cl_rm': 0.64, 'w_air_ls': 1.64, 'w_gy_ls': 0.05, 'w_fly_ls': 0.33, 'w_cl_ls': 0.71, 'beta_phk': 2.97, 'eta_plant': 0.6, 'gcv': 25.0, 'eta_kiln': 0.95, 'mol_wt_co2': 44.0, 'mol_wt_c': 12.0}


In [ ]:
# Creating dictionary for variables and their values
static_var_dict = dict(zip(static_var_df['variables'],static_var_df['values']))
print(static_var_dict)

{'m_ls': 100}


In [ ]:
# Substituting parameters in equation with their values
modified_eq_list_param = []
for eq in eq_list:
  for key in param_dict:
    if key in eq:
        value = param_dict.get(key)
        eq = eq.replace(key, str(param_dict.get(key)))
  modified_eq_list_param.append(eq)
print(modified_eq_list_param)

[' (1 + 0.11) * m_ls - m_rm', ' m_rm - m_rawmeal', ' (2.97 * 0.71 / 25.0) * (m_ls / 0.6) - m_pcoal', ' m_cooler_air - 0.76 * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas', ' 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement', ' (m_ls * 44.0 / 100) + (0.76 * 0.95 * 44.0 / 12.0) * m_pcoal - m_co2', ' e_crusher - 3.5 * m_rm', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 2.97 * 1000 * m_clinker', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [ ]:
# Substituting variables in equation with their values
modified_eq_list_var = [] # list after substituting for variables with corresponding values
for eq in modified_eq_list_param:
  for key in static_var_dict:
    if key in eq:
        value = static_var_dict.get(key)
        eq = eq.replace(key, str(static_var_dict.get(key)))
  modified_eq_list_var.append(eq)
print(modified_eq_list_var)

[' (1 + 0.11) * 100 - m_rm', ' m_rm - m_rawmeal', ' (2.97 * 0.71 / 25.0) * (100 / 0.6) - m_pcoal', ' m_cooler_air - 0.76 * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas', ' 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + 0.05 * 100 + 0.33 * 100 - m_cement', ' (100 * 44.0 / 100) + (0.76 * 0.95 * 44.0 / 12.0) * m_pcoal - m_co2', ' e_crusher - 3.5 * m_rm', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 2.97 * 1000 * m_clinker', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [ ]:
for item in modified_eq_list_var:
  print(item)

 (1 + 0.11) * 100 - m_rm
 m_rm - m_rawmeal
 (2.97 * 0.71 / 25.0) * (100 / 0.6) - m_pcoal
 m_cooler_air - 0.76 * 23.0158 * m_pcoal
 m_cooler_air - m_phk_air
 (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas
 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker
 m_hotclinker - m_clinker
 m_clinker + 0.05 * 100 + 0.33 * 100 - m_cement
 (100 * 44.0 / 100) + (0.76 * 0.95 * 44.0 / 12.0) * m_pcoal - m_co2
 e_crusher - 3.5 * m_rm
 e_rawmill - 26.0 * m_rawmeal
 e_coalmill - 76.3 * m_pcoal
 e_phk_total - 2.97 * 1000 * m_clinker
 e_cooler - 37.7 * m_hotclinker
 e_cementmill - 30.0 * m_cement


# Solving the optimization problem

In [ ]:
# Define the objective function
def objective_function(decision_variables):
    # x, y, z, w = initial_guess
    for index, variable in enumerate(decision_var_list):
      globals()[variable] = decision_variables[index]

    equations = []
    for equation in modified_eq_list_var:
      equations.append(eval(equation))
    # print(equations)
    squared_errors = [result**2 for result in equations]
    return sum(squared_errors)

In [ ]:
def constraints(decision_variables):

    for index, variable in enumerate(decision_var_list):
      globals()[variable] = decision_variables[index]

    constraints = []
    for equation in modified_eq_list_var:
      constraints.append(eval(equation))
    return constraints

In [ ]:
# Initial guess for the decision variables
initial_guess = [1] * len(decision_var_list)

# Bounds for the decision variables
# bounds = [(0, 1000)]* len(variables_list)

# If type of decision variable is correctable the bounds are adjusted to a tolerance
tol = 0.1
bounds = []
for var in decision_var_list:
  if var in correct_vars:
    value = correct_vars_df.loc[correct_vars_df['parameters'] == var, 'values'].values[0]
    # value = param_dict[var]
    lower_bound = value * (1 - tol)
    upper_bound = value * (1 + tol)
    bounds.append((lower_bound, upper_bound))
  else:
    bounds.append((None, None))

result = minimize(objective_function, initial_guess, bounds = bounds, constraints={'type': 'eq', 'fun': constraints})

In [ ]:
# Extract the optimal solution
optimal_solution = result.x
print("Optimal solution:", optimal_solution)

Optimal solution: [1.11000000e+02 1.11000000e+02 1.40580000e+01 2.45902648e+02
 7.44139200e+01 2.96546728e+02 7.44139200e+01 1.12413920e+02
 2.45902648e+02 3.88500000e+02 2.88600000e+03 1.07262540e+03
 2.21009342e+05 2.80540478e+03 3.37241760e+03 8.12162120e+01
 1.00000000e+00 6.60000000e-01]


# Print solution

In [ ]:
for variable, value in zip(decision_var_list, optimal_solution):
    value = round(value,2)
    print(f"{variable}: {value}")

m_rm: 111.0
m_rawmeal: 111.0
m_pcoal: 14.06
m_phk_air: 245.9
m_hotclinker: 74.41
m_stackgas: 296.55
m_clinker: 74.41
m_cement: 112.41
m_cooler_air: 245.9
e_crusher: 388.5
e_rawmill: 2886.0
e_coalmill: 1072.63
e_phk_total: 221009.34
e_cooler: 2805.4
e_cementmill: 3372.42
m_co2: 81.22
eta_kiln: 1.0
eta_plant: 0.66
